In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Reshape, add
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from tensorflow.keras import regularizers
from skimage.util import random_noise
import cv2
import keras
import os
from PIL import Image
import datetime
from tensorflow.keras.callbacks import TensorBoard
from keras.models import model_from_json

In [2]:
PATH=r'B:\Major\optimumDataset\clean'
PATH2=r'B:\Major\optimumDataset\smudged'
directory=os.listdir(PATH)
directory2=os.listdir(PATH2)
directory.sort()
directory2.sort()

In [3]:
X_train=[]
for file in directory:
    img=cv2.imread("{}/{}".format(PATH,file),0)
    # print(img)
    try:
      # img=cv2.resize(img)
      X_train.append(img)
    except:
      pass

In [4]:
X_train=np.array(X_train)
X_train.shape

(1044, 256, 256)

In [5]:
y_train=[]
for file in directory2:
    # print(file)
    img=cv2.imread("{}/{}".format(PATH2,file),0)
    # print(img)
    try:
      # img=cv2.resize(img)
      y_train.append(img)
    except:
      pass

In [6]:
y_train=np.array(y_train)
y_train.shape

(1044, 256, 256)

In [7]:
X_train = X_train/255.0
y_train=y_train/255.0

In [8]:
shape=X_train.shape

In [9]:
X_train=X_train.reshape(shape[0],shape[1],shape[2],1)
y_train=y_train.reshape(shape[0],shape[1],shape[2],1)

In [10]:
layers=[2,3]
nodes=[32,64,128]

In [11]:
for l in layers:
  model=Sequential()
  input_img = keras.Input(shape=(256, 256, 1))
    #encoder
  for n1 in range(0,3):
    x = Conv2D(nodes[n1], (3, 3), activation='relu', padding='same')(input_img)
    x = MaxPooling2D((2, 2), padding='same')(x)
    # print(nodes[n1])
    for n2 in range(0,3):
      y = Conv2D(nodes[n2], (3, 3), activation='relu', padding='same')(x)
      y = MaxPooling2D((2, 2), padding='same')(y)
      if l==3:
        for n3 in range(0,3):
          z = Conv2D(nodes[n3], (3, 3), activation='relu', padding='same')(y)
          z = MaxPooling2D((2, 2), padding='same')(z)
          z = Conv2D(nodes[n3], (3, 3), activation='relu', padding='same')(z)
          z = UpSampling2D((2, 2))(z)
          z = Conv2D(nodes[n2], (3, 3), activation='relu', padding='same')(z)
          z = UpSampling2D((2, 2))(z)
          z = Conv2D(nodes[n1], (3, 3), activation='relu', padding='same')(z)
          z = UpSampling2D((2, 2))(z)
          decoded = Conv2D(1, (3, 3), activation='sigmoid',padding='same')(z)

          autoencoder = keras.Model(input_img, decoded)
          autoencoder.compile(optimizer='adamax', loss='binary_crossentropy')
          autoencoder.summary()
          autoencoder.fit(y_train[350:500], X_train[350:500],epochs=5,batch_size=16,shuffle=True)
      else:
        z = Conv2D(nodes[n2], (3, 3), activation='relu', padding='same')(y)
        z = UpSampling2D((2, 2))(z)
        a = Conv2D(nodes[n1], (3, 3), activation='relu', padding='same')(z)
        a = UpSampling2D((2, 2))(a)
        decoded = Conv2D(1, (3, 3), activation='sigmoid',padding='same')(a)

        autoencoder = keras.Model(input_img, decoded)
        autoencoder.compile(optimizer='adamax', loss='mse')
        autoencoder.summary()
        autoencoder.fit(y_train[350:500], X_train[350:500],epochs=5,batch_size=16,shuffle=True)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 1)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 32)      320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 32)        9248      
_________________________________________________________________
up_sampling2d (UpSampling2D) (None, 128, 128, 32)      0     

In [12]:
print(".")

.
